# Dash DataTable Tutorial

This notebook will guide us through various features of **Dash DataTables** using the Dash framework. We'll cover:

1. Creating a simple table.
2. Styling and customizing the table.
3. Conditional styling.
4. Sorting, filtering, and pagination.
5. Adding interactivity like sorting, filtering, and editing.
6. Conditional formatting.
7. Editable columns and dropdowns.
8. Row selectability and exporting tables.
9. Dynamically updating tables using Dash callbacks.

---

### Step 1: Installing Required Libraries

First, install Dash, Dash Bootstrap Components, and Pandas if you haven't already:

```bash
!pip install dash dash-bootstrap-components pandas dash-table yfinance


In [1]:
# Importing necessary libraries
import dash
import dash_bootstrap_components as dbc
import pandas as pd
from dash import html, Input, Output
from dash.dash_table import DataTable, FormatTemplate, Format
import yfinance as yf
from datetime import datetime as dt
import string


In [2]:
# Fetch data for MSFT (Microsoft)
msft = yf.Ticker("MSFT")
df = msft.history(period="max")

# Reset the index and format the Date column
df = df.reset_index()
df['Date'] = df['Date'].map(dt.date)
df['Close'] = df['Close'].astype(float)

# For demo purposes, we'll show only a subset of the data
df = df.iloc[1:10]


In [3]:
def create_basic_table(df):
    # Basic DataTable with no customization
    table = DataTable(
        data=df.to_dict('records'),
        columns=[{'id': c, 'name': c} for c in df.columns],
        style_table={'border': 'none'},
        style_cell={
            'border': 'none',
            'fontSize': '12px', 
            'boxShadow': '0 0',
            'height': 'auto',
            'width': 'auto',
            'whiteSpace': 'normal',
            'textAlign': 'center'
        },
        style_header={
            'border': 'none',
            'backgroundColor': 'white',
            'fontWeight': 'bold',
            'borderBottom': '1px solid black',
            'font-size': '14px'
        }
    )
    return table


In [4]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = html.Div(
    [
        html.H1("Dash DataTable in Jupyter Notebook"),
        html.Div(create_basic_table(df), id='table-container'),
    ]
)

# Run the app in the notebook
app.run_server(debug=True, use_reloader=False, port=8051)
df.head(3)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
1,1986-03-14,0.059827,0.063032,0.059827,0.061963,308160000,0.0,0.0
2,1986-03-17,0.061963,0.063566,0.061963,0.063032,133171200,0.0,0.0
3,1986-03-18,0.063032,0.063566,0.060895,0.061429,67766400,0.0,0.0


In [5]:
def create_styled_table(df):
    table = DataTable(
        data=df.to_dict('records'),
        columns=[{'id': c, 'name': c} for c in df.columns],
        style_table={'border': '1px solid black'},
        style_cell={
            'border': '1px solid #ccc', #
            'fontSize': '12px',
            'height': 'auto',
            'width': 'auto',
            'whiteSpace': 'normal',
            'textAlign': 'center'
        },
        style_header={
            'backgroundColor': '#f1f1f1', #
            'fontWeight': 'bold',
            'font-size': '14px',
            'border': '1px solid #ddd' #
        },
        style_data={ ##
            'backgroundColor': 'white',
            'border': '1px solid #ddd',
            'height': 'auto'
        }           ##
    )
    return table


In [6]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = html.Div(
    [
        html.H1("Dash DataTable in Jupyter Notebook"),
        html.Div(create_styled_table(df), id='table-container'),
    ]
)

# Run the app in the notebook
app.run_server(debug=True, use_reloader=False, port=8052)
df.head(3)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
1,1986-03-14,0.059827,0.063032,0.059827,0.061963,308160000,0.0,0.0
2,1986-03-17,0.061963,0.063566,0.061963,0.063032,133171200,0.0,0.0
3,1986-03-18,0.063032,0.063566,0.060895,0.061429,67766400,0.0,0.0


In [7]:
def create_conditionally_styled_table(df):
    cond_style = [
        {
            'if': {'column_id': 'Close'},
            'color': 'blue',
            'fontWeight': 'bold',
        },
        {
            'if': {'column_id': 'Volume'},
            'backgroundColor': '#eaf1f8',
            'color': 'red'
        }
    ]

    table = DataTable(
        data=df.to_dict('records'),
        columns=[{'id': c, 'name': c} for c in df.columns],
        style_table={'border': 'none'},
        style_cell_conditional=cond_style, #
        style_cell={'border': 'none', 'fontSize': '12px', 'height': 'auto'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
    )
    return table


In [8]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = html.Div(
    [
        html.H1("Dash DataTable in Jupyter Notebook"),
        html.Div(create_conditionally_styled_table(df), id='table-container'),
    ]
)

# Run the app in the notebook
app.run_server(debug=True, use_reloader=False, port=8053)
df.head(3)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
1,1986-03-14,0.059827,0.063032,0.059827,0.061963,308160000,0.0,0.0
2,1986-03-17,0.061963,0.063566,0.061963,0.063032,133171200,0.0,0.0
3,1986-03-18,0.063032,0.063566,0.060895,0.061429,67766400,0.0,0.0


In [9]:
def create_sort_filter_paginated_table(df):
    df['Alphabet'] = list(string.ascii_lowercase)[:df.shape[0]]
    table = DataTable(
        data=df.to_dict('records'),
        columns=[{'id': c, 'name': c} for c in df.columns],
        sort_action='native',    # Enable sorting
        filter_action='native',  # Enable filtering
        page_action='native',    # Enable pagination
        page_size=5,             # Number of rows per page
        style_table={'border': 'none'},
        style_cell={'border': 'none', 'fontSize': '12px', 'height': 'auto'}
    )
    return table


In [10]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = html.Div(
    [
        html.H1("Dash DataTable in Jupyter Notebook"),
        html.Div(create_sort_filter_paginated_table(df), id='table-container'),
    ]
)

# Run the app in the notebook
app.run_server(debug=True, use_reloader=False, port=8054)
df.head(3)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Alphabet
1,1986-03-14,0.059827,0.063032,0.059827,0.061963,308160000,0.0,0.0,a
2,1986-03-17,0.061963,0.063566,0.061963,0.063032,133171200,0.0,0.0,b
3,1986-03-18,0.063032,0.063566,0.060895,0.061429,67766400,0.0,0.0,c


In [14]:
def create_formatted_table(df):
    cond_format = {
        'commas': ['Volume'],
        'decimals': ['Open', 'High', 'Low'],
        'money': ['Close']
    }

    formats = {
        **{m: FormatTemplate.money(3) for m in cond_format['money']},
        **{c: Format.Format(group=',') for c in cond_format['commas']},
        **{c: Format.Format(precision=4, scheme=Format.Scheme.fixed) for c in cond_format['decimals']}
    }

    columns = [
        {'id': c, 'name': c, 'editable': True, 'type': 'numeric', 'format': formats.get(c, None)} if c in formats else {'id': c, 'name': c}
        for c in df.columns
    ]
    print(formats['Close'])
    table = DataTable(
        data=df.to_dict('records'),
        columns=columns,
        style_table={'border': 'none'},
        style_cell={'border': 'none', 'fontSize': '12px', 'height': 'auto'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'},
        style_data={'whiteSpace': 'pre-line', 'height': 'auto'}
    )
    return table


In [17]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = html.Div(
    [
        html.H1("Dash DataTable in Jupyter Notebook"),
        html.Div(create_formatted_table(df), id='table-container'),
    ]
)

# Run the app in the notebook
app.run_server(debug=True, use_reloader=False, port=8055)

In [3]:
def create_editable_table(df):
    columns = [
        {
            'id': c, 
            'name': c, 
            'editable': True, 
            'presentation': 'dropdown' if c == 'Dividends' else 'input'
        }
        for c in df.columns
    ]

    table = DataTable(
        data=df.to_dict('records'),
        columns=columns,
        style_table={'border': 'none'},
        style_cell={'border': 'none', 'fontSize': '12px', 'height': 'auto'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'},
        style_data={'whiteSpace': 'pre-line', 'height': 'auto'},
    )
    return table

In [4]:
def create_selectable_and_exportable_table(df):
    table = DataTable(
        data=df.to_dict('records'),
        columns=[{'id': c, 'name': c} for c in df.columns],
        row_selectable='single',  # Allows row selection
        export_format="xlsx",  # Enable exporting as Excel
        style_table={'border': 'none'},
        style_cell={'border': 'none', 'fontSize': '12px', 'height': 'auto'},
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'}
    )
    return table


In [5]:
# Create the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = html.Div(
    [
        html.H1("Dash DataTable Tutorial"),
        
        dbc.Checklist(
            options=[{"label": "Switch Table Type", "value": 1}],
            value=[],
            id="format",
            inline=True,
            switch=True,
        ),
        
        html.Br(),
        
        html.Div(id="table-container")
    ]
)

# Callback to dynamically update the table
@app.callback(
    Output("table-container", "children"),
    Input("format", "value"),
)
def update_table(format_option):
    if 1 in format_option:
        return create_editable_table(df)  # Create editable table with dropdowns when selected
    else:
        return create_selectable_and_exportable_table(df)  # Create basic table when not selected

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8056)
